In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [49]:
traindata=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
testdata=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [4]:
traindata.iloc[50]

id                                                         73
keyword                                                ablaze
location                             Sheffield Township, Ohio
text        Deputies: Man shot before Brighton home set ab...
target                                                      1
Name: 50, dtype: object

In [5]:
traindata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [6]:
traindata['keyword'].unique()

array([nan, 'ablaze', 'accident', 'aftershock', 'airplane%20accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
       'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
       'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard', 'blood',
       'bloody', 'blown%20up', 'body%20bag', 'body%20bagging',
       'body%20bags', 'bomb', 'bombed', 'bombing', 'bridge%20collapse',
       'buildings%20burning', 'buildings%20on%20fire', 'burned',
       'burning', 'burning%20buildings', 'bush%20fires', 'casualties',
       'casualty', 'catastrophe', 'catastrophic', 'chemical%20emergency',
       'cliff%20fall', 'collapse', 'collapsed', 'collide', 'collided',
       'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew',
       'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris',
       'deluge', 'deluged', 'demolish', 'demolished', 'demolition',
       'derail', 'der

In [7]:

ytrain=traindata['target']
traindata.drop(['keyword','id','location','target'],axis=1,inplace=True)

In [8]:
traindata.text

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [17]:
tokenizer=keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(traindata.text)
tokenlen=len(tokenizer.word_index)+1

In [25]:
tokenizer

In [26]:
xtrain=keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(traindata.text),maxlen=30)

In [21]:
len(xtrain)

7613

In [44]:
model=keras.models.Sequential()
model.add(keras.layers.Embedding(tokenlen,300,input_length=30))
model.add(keras.layers.SpatialDropout1D(0.5))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(16,dropout=0.5,recurrent_dropout=0.5)))
model.add(keras.layers.Dense(1,activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),loss='binary_crossentropy',metrics=['accuracy'])


In [45]:
callback = keras.callbacks.ReduceLROnPlateau(factor=0.1,
                    min_lr = 0.01,
                    monitor = 'val_loss',
                    verbose = 1)
his=model.fit(xtrain,ytrain,batch_size=512,epochs=6,validation_split=0.2,callbacks=[callback])

Epoch 1/6
12/12 [==============================] - 10s 457ms/step - loss: 0.6852 - accuracy: 0.5380 - val_loss: 0.6816 - val_accuracy: 0.5364
Epoch 2/6
12/12 [==============================] - 5s 419ms/step - loss: 0.6578 - accuracy: 0.5838 - val_loss: 0.6383 - val_accuracy: 0.6507
Epoch 3/6
12/12 [==============================] - 5s 416ms/step - loss: 0.6137 - accuracy: 0.6942 - val_loss: 0.5940 - val_accuracy: 0.7065
Epoch 4/6
12/12 [==============================] - 5s 415ms/step - loss: 0.5469 - accuracy: 0.7401 - val_loss: 0.5238 - val_accuracy: 0.7551
Epoch 5/6
12/12 [==============================] - 5s 415ms/step - loss: 0.4288 - accuracy: 0.8282 - val_loss: 0.4764 - val_accuracy: 0.7735
Epoch 6/6
12/12 [==============================] - 5s 414ms/step - loss: 0.3146 - accuracy: 0.8755 - val_loss: 0.4709 - val_accuracy: 0.7905


In [50]:
testid=testdata['id']
testdata.drop(['keyword','id','location'],axis=1,inplace=True)
tokenizer=keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(testdata.text)
xtest=keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(testdata.text),maxlen=30)

In [59]:
pre=model.predict(xtest)
pre=list(1 if x>0.5 else 0 for x in list(pre))

In [61]:
res=pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')
#result=pd.DataFrame((pre,testid))
res['target']=pre


In [63]:
testid

0           0
1           2
2           3
3           9
4          11
        ...  
3258    10861
3259    10865
3260    10868
3261    10874
3262    10875
Name: id, Length: 3263, dtype: int64

In [65]:
res.to_csv('res.csv',index=False)